# Basic Deep Learning Face Recogntion
## Building a Friends TV Show Character Identifier

## The learning objective of this lesson (25.1) is the create a 'dumb' face classifer using our LittleVGG model. We are simply training it with 100s of pictures of each Friends Character, and testing our model using a Test Video. 

## You will see how this is an in-effective way to do Face Recognition, why?
Because a traditional CNN will be looking at small subsections of a face to identify it. However, the angle and deformation of a face can easily throw off our model, making it mis-classify and match it to faces that aren't correct - this will happen a lot especially when our training data looks different to our test data. Imagine if the training data, one face was titled downward mostly. Our CNN will more likely identify any future face on our test data that looks downward, to be that face.

### Let's train our model
I've created a dataset with the faces of 4 Friends characters taken from a handful of different scenes.

In [18]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [4]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
import os

num_classes = 4
img_rows, img_cols = 220, 220
batch_size = 8

train_data_dir = 'E:/25. Face Recognition/faces/train'
validation_data_dir = 'E:/25. Face Recognition/faces/validation'

# Let's use some data augmentaiton 
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      shear_range=0.3,
      zoom_range=0.3,
      width_shift_range=0.4,
      height_shift_range=0.4,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)
 
validation_generator = validation_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)

Found 2663 images belonging to 4 classes.
Found 955 images belonging to 4 classes.


In [5]:
#Our Keras imports
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation, Flatten, Dropout, Dense

### Creating a simple VGG based model for Face Recognition

In [10]:
model = Sequential()

model.add(Conv2D(16, (3, 3), padding = 'same', kernel_initializer="he_normal",
                 input_shape = (img_rows, img_cols, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(16, (3, 3), padding = "same", kernel_initializer="he_normal", 
                 input_shape = (img_rows, img_cols, 3)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))


# Block #1: second CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #2: second CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))



# Block #3: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# Block #4: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))



# Block #vanish: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(512, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(512, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))


# Block #vanish: third CONV => RELU => CONV => RELU => POOL
# layer set
model.add(Conv2D(1024, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(1024, (3, 3), padding="same", kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))


# Block #5: first set of FC => RELU layers
model.add(Flatten())
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #6: second set of FC => RELU layers
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block #7: softmax classifier
model.add(Dense(num_classes, kernel_initializer="he_normal"))
model.add(Activation("softmax"))

print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 220, 220, 16)      448       
_________________________________________________________________
activation_63 (Activation)   (None, 220, 220, 16)      0         
_________________________________________________________________
batch_normalization_60 (Batc (None, 220, 220, 16)      64        
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 220, 220, 16)      2320      
_________________________________________________________________
activation_64 (Activation)   (None, 220, 220, 16)      0         
_________________________________________________________________
batch_normalization_61 (Batc (None, 220, 220, 16)      64        
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 110, 110, 16)     

### Training our Model

In [ ]:
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

                     
checkpoint = ModelCheckpoint("E:/25. Face Recognition/face_recognition_friends_vgg.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 15,
                          verbose = 2,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.4, patience = 1, verbose = 1, min_delta = 0.00000000000000000000000000001)

# we put our call backs into a callback list
callbacks = [checkpoint, reduce_lr]

# We use a very small learning rate 
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics = ['accuracy'])

nb_train_samples = 2663
nb_validation_samples = 955
epochs = 70

history = model.fit_generator(
    train_generator,
    steps_per_epoch = nb_train_samples // batch_size,
    epochs = epochs,
    callbacks = callbacks,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size)

Epoch 1/70
332/332 [==============================] - ETA: 0s - loss: 1.5833 - accuracy: 0.4580
Epoch 00001: val_loss improved from inf to 1.70934, saving model to E:/25. Face Recognition\face_recognition_friends_vgg.h5
332/332 [==============================] - 39s 116ms/step - loss: 1.5833 - accuracy: 0.4580 - val_loss: 1.7093 - val_accuracy: 0.4611
Epoch 2/70
332/332 [==============================] - ETA: 0s - loss: 1.1045 - accuracy: 0.5902
Epoch 00002: val_loss improved from 1.70934 to 1.53440, saving model to E:/25. Face Recognition\face_recognition_friends_vgg.h5
332/332 [==============================] - 38s 116ms/step - loss: 1.1045 - accuracy: 0.5902 - val_loss: 1.5344 - val_accuracy: 0.3718
Epoch 3/70
332/332 [==============================] - ETA: 0s - loss: 0.8424 - accuracy: 0.6863
Epoch 00003: val_loss did not improve from 1.53440

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0004000000189989805.
332/332 [==============================] - 35s 105ms/step - 

In [16]:
validation_generator.class_indices

{'Chandler': 0, 'Joey': 1, 'Pheobe': 2, 'Rachel': 3}

In [17]:
validation_generator.class_indices.items()

dict_items([('Chandler', 0), ('Joey', 1), ('Pheobe', 2), ('Rachel', 3)])

#### Getting our Class Labels

In [1]:
# Importing the libraries
from PIL import Image
from keras.applications.vgg16 import preprocess_input
import random
import cv2
from keras.models import load_model
import numpy as np

In [2]:
model = load_model('E:/25. Face Recognition/face_recognition_friends_vgg.h5')

In [3]:
face_cascade = cv2.CascadeClassifier('E:/25. Face Recognition/haarcascade_frontalface_default.xml')

In [4]:
cap = cv2.VideoCapture('E:/25. Face Recognition/testfriends.mp4')

In [5]:
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    #gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face


In [6]:
face_classes = {0: 'Chandler', 1: 'Joey', 2: 'Pheobe', 3: 'Rachel'}

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    
margin = 0.2
# load model and weights
img_size = 64

In [9]:
while True:
    ret, frame = cap.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
           #Resizing into 128x128 because we trained the model with this image size.
        img_array = np.array(im)
                    #Our keras model used a 4D tensor, (images x height x width x channel)
                    #So changing dimension 128x128x3 into 1x128x128x3 
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)
       
                     
        name="None matching"

        if(pred[0][0]>0.5):
            name='Chandler'
            cv2.putText(img_array,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        elif(pred[0][1]>0.5):
            name='Joey'
            cv2.putText(img_array,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        elif(pred[0][2]>0.5):
            name='Pheobe'
            cv2.putText(img_array,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        elif(pred[0][3]>0.5):
            name='Rachel'
            cv2.putText(img_array,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()   

[[0.39399692 0.6060031  0.         0.        ]]
[[0.45889425 0.54110575 0.         0.        ]]
[[0.4257712  0.57422876 0.         0.        ]]
[[0.40944582 0.5905542  0.         0.        ]]
[[0.41158932 0.5884106  0.         0.        ]]
[[4.9645808e-01 5.0354189e-01 1.6220768e-38 0.0000000e+00]]
[[0.2804793 0.7195207 0.        0.       ]]
[[4.8221859e-01 5.1778138e-01 1.6996897e-38 0.0000000e+00]]
[[0.58481216 0.41518778 0.         0.        ]]
[[0.5799042 0.4200958 0.        0.       ]]
[[5.9426618e-01 4.0573385e-01 3.5481471e-38 0.0000000e+00]]
[[5.0072002e-01 4.9927998e-01 1.2990014e-38 0.0000000e+00]]
[[4.6058649e-01 5.3941351e-01 1.5550261e-38 0.0000000e+00]]
[[4.4924632e-01 5.5075365e-01 1.5810933e-38 0.0000000e+00]]
[[4.4920006e-01 5.5079991e-01 1.5777074e-38 0.0000000e+00]]
[[0.51954514 0.4804549  0.         0.        ]]
[[5.9413278e-01 4.0586719e-01 1.7146802e-38 0.0000000e+00]]
[[5.6800491e-01 4.3199503e-01 1.3807762e-38 0.0000000e+00]]
[[0.5472599  0.45274007 0.         0

### Testing our model on some real video